In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset, random_split
from week5_LSTM_model import LSTMModel, load_data


# Training function
def train_model(model, train_loader, val_loader, optimizer, criterion, epochs=100):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                output = model(X_batch)
                loss = criterion(output, y_batch)
                val_loss += loss.item()
        
        # if (epoch + 1) % 10 == 0:
        #     print(f'Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, '
        #           f'Validation Loss: {val_loss/len(val_loader):.4f}')

def eval_model(model,criterion,val_loader):
    model.eval()
    train_loss = 0
    for X_batch, y_batch in val_loader:
        # optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        train_loss += loss.item()

    return train_loss/len(val_loader)
# Testing and visualization



In [3]:
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm
# Parameter grid
params_search = {
    'hidden_size': [16, 32, 64],
    'num_layers': [1,2,3],
    'batch_size':[16,32,64],
    'seq_length':[12,24],
    'lr':[0.01,0.001]
}

# Generate all combinations of parameters
param_grid = ParameterGrid(params_search)

# Loop through each combination
losses = []
for params in tqdm(param_grid):
    # print(param_combination)
    'a'
    # Access individual parameters like this
    train_data, val_data, test_data, scaler = load_data('datasets_56102_107707_monthly-beer-production-in-austr.csv',params['seq_length'])


    # Create data loaders
    batch_size = params['batch_size']
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size)
    test_loader = DataLoader(test_data, batch_size=batch_size)



    # Initialize the model, loss function, and optimizer
    model = LSTMModel(hidden_size=params['hidden_size'],num_layers=params['num_layers'])
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'])



    # Train the model
    train_model(model, train_loader, val_loader, optimizer, criterion, epochs=100)

    losses.append(eval_model(model,criterion,val_loader))
    
    # You can now train your model with each combination
    # model = YourModel(nodes=nodes, lr=lr)
    # model.fit(X_train, y_train)


100%|██████████| 108/108 [13:22<00:00,  7.43s/it]


In [4]:
best_model_ind = losses.index(min(losses))
best_model_params = param_grid[best_model_ind]

print(best_model_params)

{'seq_length': 24, 'num_layers': 1, 'lr': 0.01, 'hidden_size': 64, 'batch_size': 32}
